In [1]:
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
df = pd.read_csv('data\data.csv')
df

,Region,Country,Name,Industry,Value,Category,Year
0,Oceania,Australia,Amcor Ltd,Industrials,1779853,Equity,1998
1,Oceania,Australia,AMP Ltd,Financials,8662515,Equity,1998
2,Oceania,Australia,Ashton Mining,Basic Materials,75850,Equity,1998
3,Oceania,Australia,Australian Foundation Investment Co Ltd,Financials,793252,Equity,1998
4,Oceania,Australia,Australian Gas Light Company,Utilities,1509918,Equity,1998
...,...,...,...,...,...,...,...
189336,North America,United States,WPC Eurobond BV,Corporate Bonds,79539339,Fixed Income,2022
189337,North America,United States,Wyeth LLC,Corporate Bonds,37798334,Fixed Income,2022
189338,North America,United States,XLIT Ltd,Corporate Bonds,13322191,Fixed Income,2022
189339,North America,United States,Zimmer Biomet Holdings Inc,Corporate Bonds,40329494,Fixed Income,2022


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189341 entries, 0 to 189340
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Region    189341 non-null  object
 1   Country   189341 non-null  object
 2   Name      189341 non-null  object
 3   Industry  189341 non-null  object
 4   Value     189341 non-null  int64 
 5   Category  189341 non-null  object
 6   Year      189341 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 10.1+ MB
